<a href="https://colab.research.google.com/github/NathaliaRivadeneira/Proyecto-inteligencia-artificial/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROYECTO FINAL DE INTELIGENCIA ARTIFICIAL**


**JUAN DAVID VELÁSQUEZ R.**


 **NATHALIA RIVADENEIRA**

In [ ]:
'''IMPORTACIÓN DE TODOS LOS PAQUETES RELACIONADOS CON LOS MÉTODOS DE 
CLASIFICACIÓN Y DE EVALUACIÓN DE DICHOS MÉTODOS.
'''
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn import svm
import warnings
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.neural_network import MLPClassifier
import time

# AJUSTE DE LOS PARÁMETROS DEL ESCALIZADOR MEDIANTE EL MÉTODO MIN_MAX A UN
# RANGO DE -1 A 1.
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
#  GUARDADO DE LOS DATOS EN UN DATAFRAME DE PANDAS.
datos=pd.read_csv("Airplane_Complete_Imputation.csv",sep=",")
# SE ELIMINAN ALGUNAS COLUMNAS DE POCA RELEVANCIA Y QUE NO SE PUEDEN ASOCIAR
# A UN DATO NUMÉRICO. 
datos = datos.drop(['Model','Company','TP mods','THR','SHP','Hmax (One)','ROC (One)','Multi Engine'], axis=1)
# SE ELIMINAN TODAS LAS FILAS CON DATOS NaN.
datos = datos.dropna()

# SE ASOCIA EL GÉNERO MASCULINO COMO '0' Y EL FEMENINO COMO '1'.
datos['Engine Type'] = datos['Engine Type'].replace(['Piston'],'0')
datos['Engine Type'] = datos['Engine Type'].replace(['piston'],'0')
datos['Engine Type'] = datos['Engine Type'].replace(['Propjet'],'1')
datos['Engine Type'] = datos['Engine Type'].replace(['PT6A-21'],'1')
datos['Engine Type'] = datos['Engine Type'].replace(['Jet'],'2')

# GUARDADO DE LAS CARACTERÍSTICAS EN UNA VARIABLE.
X = datos[['Length','Height','Wing Span','FW','MEW','AUW','Vmax','Vcruise','Vstall','Hmax','ROC','Vlo','Slo','Vl','Sl','Range']]
X = np.array(X)

Y = datos[['Engine Type']]

# VARIABLE ASOCIADA AL NÚMERO DE CARACTERÍSTICAS DESEADAS.
number_of_X = 16

display(datos)
# SE GUARDA EL DATASET EN UN CSV YA ORGANIZADO Y LIMPIADO.
# RECORDAR QUE QUEDA DISPONIBLE EN LA CARPETA DE ARCHIVOS DE COLAB.
Aircraft_Handbook = datos[['Length','Height','Wing Span','FW','MEW','AUW','Vmax','Vcruise','Vstall','Hmax','ROC','Vlo','Slo','Vl','Sl','Range','Engine Type']]
Aircraft_Handbook.to_csv('Aircraft_Handbook.csv',index=False)

,Engine Type,Length,Height,Wing Span,FW,MEW,AUW,Vmax,Vcruise,Vstall,Hmax,ROC,Vlo,Slo,Vl,Sl,Range
0,0,25.250000,10.250000,37.416667,241.2,1180.0,2050.0,104.0,91.0,46.0,13000.0,450.0,900.0,391.970247,1300.0,257.745075,370.0
1,0,20.583333,8.750000,36.083333,100.5,820.0,1350.0,89.0,83.0,44.0,12300.0,600.0,720.0,26.247647,800.0,225.324824,190.0
2,0,21.416667,8.583333,35.000000,127.3,810.0,1300.0,90.0,78.0,37.0,16000.0,650.0,475.0,363.139711,850.0,585.751317,210.0
3,0,21.416667,8.583333,35.000000,127.3,800.0,1300.0,88.0,78.0,37.0,13000.0,620.0,500.0,407.797297,850.0,642.046166,210.0
4,0,21.416667,8.750000,35.000000,93.8,740.0,1220.0,83.0,74.0,33.0,12500.0,370.0,632.0,297.056192,885.0,329.571813,175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,0,35.083333,14.416667,49.416667,1045.2,4255.0,6250.0,198.0,190.0,54.0,22500.0,1400.0,1210.0,970.000000,1150.0,865.000000,957.0
856,0,35.083333,14.416667,49.000000,1045.2,3850.0,6000.0,190.0,178.0,55.0,22500.0,1400.0,1250.0,1000.000000,1350.0,950.000000,957.0
857,0,35.416667,14.416667,44.083333,1005.0,3800.0,5500.0,183.0,171.0,52.0,24000.0,1700.0,1100.0,950.000000,1050.0,900.000000,1000.0
858,0,21.166667,7.000000,30.000000,221.1,1175.0,1817.0,130.0,116.0,45.0,22000.0,1310.0,1730.0,1120.000000,1585.0,785.000000,365.0


**SEPARACIÓN DEL DATASET EN 80% ENTRENAMIENTO Y 20% EN VALIDACIÓN**

In [ ]:
'''
MEDIANTE LA FUNCIÓN TRAIN TEST SPLIT, SEPARAMOS EL DATASET EN 80% ENTRENAMIENTO
Y 20 % EN VALIDACIÓN.
'''
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, random_state = 0)
# SE ENTRENA EL NORMALIZADOR CON EL CONJUNTO DE ENTRENAMIENTO (80% DE LOS DATOS)
min_max_scaler.fit(X_train)
# SE NORMALIZA EL CONJUNTO DE ENTRENAMIENTO
X_train = min_max_scaler.transform(X_train)
# SE NORMALIZA EL CONJUNTO DE VALIDACIÓN
X_test = min_max_scaler.transform(X_test)
# SE NORMALIZA EL CONJUNTO PARA PRUEBAS DE PCA.
X_pca_pruebas = min_max_scaler.transform(X_train)

**EJECUCIÓN DEL ANÁLISIS POR COMPONENTES (PCA)**

In [ ]:
'''
SE CREAN DOS CONJUNTOS DE CARACTERÍSTICAS CON Y SIN REDUCCIÓN DIMENSIONAL.
EN ESTE PRIMERO NO SE REDUCEN LAS CARACTERÍSTICAS CON EL FIN DE COMPARAR CUANTO
CAMBIA LA SUMA DE LA VARIANZA EXPLICADA AL HABER DISMINUIDO LA DIMENSIÓN DE LAS
CARACTERÍSTICAS.
'''
pca = decomposition.PCA(n_components=number_of_X,whiten=True,svd_solver='full')
pca.fit(X_train)
X_pca_pruebas = pca.transform(X_train)
print("Pesos de PCA sin reducción dimensional: \n")
print("Pesos de PCA:",pca.explained_variance_ratio_)
print("Suma de PCA:",pca.explained_variance_ratio_,sum(pca.explained_variance_ratio_),"\n")

'''
AQUÍ SE ITERA DESDE #X - 1 HASTA HALLAR UN 99.7% DE LA SUMATORIA DE LA VARIANZA
EXPLICADA, ADEMÁS, EL RESULTADO MOSTRARÁ CUANTO SE REDUJERON DIMENSIONALMENTE 
LAS CARACTERÍSTICAS.
SE VAN REDUCIENDO EL NÚMERO DE CARACTERÍSTICAS HASTA HALLAR LA V. EXPLICADA 
MENCIONADA.
'''

for i in range(1,number_of_X):
  X_pca_pruebas = X_train
  pca_pruebas = decomposition.PCA(n_components = number_of_X-i,whiten=True,svd_solver='full')
  pca_pruebas.fit(X_pca_pruebas)
  X_pca_pruebas = pca.transform(X_pca_pruebas)
  print("Para un X de",17-i,"Dimensiones:\n")
  print("Pesos de PCA con reducción dimensional de X17 a X",17-i,": \n")
  print("Pesos de PCA:",pca_pruebas.explained_variance_ratio_)
  print("Suma de PCA:\n",pca_pruebas.explained_variance_ratio_,sum(pca_pruebas.explained_variance_ratio_),"\n")

  if(sum(pca_pruebas.explained_variance_ratio_)<0.997):
    numX = i - 1
    break

'''
COMO SE OBSERVA EN LA SALIDA DE CONSOLA, SE LOGRÓ HACER UNA REDUCCIÓN DIMENSIONAL
DE 17 A 13 LOGRANDO UNA VARIANZA EXPLICADA DE 0.998, EVIDENCIADA EN LA 
APLICACIÓN DE PCA DEL CÓDIGO A CONTINUACIÓN.
'''

pca = decomposition.PCA(n_components = number_of_X - numX,whiten=True,svd_solver='full')
pca.fit(X_train)
X_train = pca.transform(X_train)
print("Pesos de PCA con la reducción dimensional Final: \n")
print("Pesos de PCA:",pca.explained_variance_ratio_)
print("Suma de PCA:",pca.explained_variance_ratio_,sum(pca.explained_variance_ratio_),"\n")
print("Finalmente se escoje una reducción dimensional de 17 a", number_of_X-numX + 1)

X_test = pca.transform(X_test)
  

Pesos de PCA sin reducción dimensional: 

Pesos de PCA: [8.28698691e-01 7.43428995e-02 2.67529653e-02 1.61839330e-02
 1.23254377e-02 1.14550403e-02 7.40031622e-03 6.92197600e-03
 5.02699834e-03 3.26336875e-03 2.73038224e-03 1.54635683e-03
 1.42615908e-03 9.99989462e-04 8.63573876e-04 6.19123879e-05]
Suma de PCA: [8.28698691e-01 7.43428995e-02 2.67529653e-02 1.61839330e-02
 1.23254377e-02 1.14550403e-02 7.40031622e-03 6.92197600e-03
 5.02699834e-03 3.26336875e-03 2.73038224e-03 1.54635683e-03
 1.42615908e-03 9.99989462e-04 8.63573876e-04 6.19123879e-05] 1.0 

Para un X de 16 Dimensiones:

Pesos de PCA con reducción dimensional de X17 a X 16 : 

Pesos de PCA: [0.82869869 0.0743429  0.02675297 0.01618393 0.01232544 0.01145504
 0.00740032 0.00692198 0.005027   0.00326337 0.00273038 0.00154636
 0.00142616 0.00099999 0.00086357]
Suma de PCA:
 [0.82869869 0.0743429  0.02675297 0.01618393 0.01232544 0.01145504
 0.00740032 0.00692198 0.005027   0.00326337 0.00273038 0.00154636
 0.00142616 0.000


**PRUEBAS CON 5 DIFERENTES CLASIFICADORES UTILIZANDO EL MÉTODO GRIDSEARCH CV**

**REGRESIÓN LOGÍSTICA**

In [ ]:
# SELECCIÓN DE HIPERPARÁMETROS DEL CLASIFICADOR. ESTOS SERÁN LOS QUE SE VAN A 
# ITERARSE.
parameters = {'penalty':('l2','l1'), 'C':[0.01, 0.03, 0.1, 0.3, 1, 2, 5, 10 ,30, 100]}

# SE GUARDA EL TIPO DE CLASIFICADOR EN UNA VARIABLE.
log_reg=LogisticRegression()
logreg_cv=GridSearchCV(log_reg, parameters, cv= 5, refit = 'True', verbose = 0)

# ENTRENAMIENTO DEL MODELO CON MEJORES RESULTADOS.
start = time.time()
logreg_cv.fit(X_train, Y_train)
stop = time.time()
print(f"Training time: {stop - start}s")
sorted(logreg_cv.cv_results_.keys())

# IMPRESIÓN DE LOS RESULTADOS PARA CADA CLASE Y LAS CARACTERÍSTICAS DEL MODELO
# CON MEJORES HIPERPARÁMETROS.
print("Mejores Hiperparámetros:", logreg_cv.best_params_)
print("\n")

'''
FINALMENTE SE REALIZA TODO EL PROCESO DE EVALUACIÓN DEL CLASIFICADOR APLICADO 
QUE EN ESTE CASO, ES REGRESIÓN LOGÍSTICA.
IMPLEMENTANDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
'''
y_test_predicted = logreg_cv.predict(X_test)
y_test_scores = logreg_cv.decision_function(X_test)
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)

# IMPRESIÓN DE UN DATAFRAME CON LOS RESULTADOS.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")

Training time: 1.365471601486206s
Mejores Hiperparámetros: {'C': 100, 'penalty': 'l2'}


Resultado mediante MCC: 0.8928944952965416
Resultado mediante Accuracy 0.9534883720930233
Reporte de resultados en cada clase:



,0,1,2,macro avg,weighted avg
precision,0.968000,0.75,1.000000,0.906000,0.953674
recall,0.975806,0.75,0.968750,0.898185,0.953488
f1-score,0.971888,0.75,0.984127,0.902005,0.953524
support,124.000000,16.00,32.000000,172.000000,172.000000


**MÁQUINAS DE SOPORTE VECTORIAL (SVM)**

In [ ]:
# SELECCIÓN DE HIPERPARÁMETROS DEL CLASIFICADOR. ESTOS SERÁN LOS QUE SE VAN A 
# ITERARSE.
parameters = {'kernel':('poly', 'rbf','sigmoid'), 'C':[0.01, 0.03, 0.1, 0.3, 1, 2, 5, 10 ,30,100], 'decision_function_shape': ('ovo', 'ovr')}

# SE GUARDA EL TIPO DE CLASIFICADOR EN UNA VARIABLE.
svc = svm.SVC()

# ESTA SECCIÓN HACE LA BÚSQUEDA POR GRADIENTE PARTIENDO DE LOS HIPERPARÁMETROS DADOS.
clf = GridSearchCV(svc, parameters, cv = 5, refit = 'True', verbose = 0)

# ENTRENAMIENTO DEL MODELO CON MEJORES RESULTADOS.
start = time.time()
clf.fit(X_train, Y_train)
stop = time.time()
print(f"Training time: {stop - start}s")
sorted(clf.cv_results_.keys())

# IMPRESIÓN DE LOS RESULTADOS PARA CADA CLASE Y LAS CARACTERÍSTICAS DEL MODELO
# CON MEJORES HIPERPARÁMETROS.
print("Mejores Hiperparámetros:", clf.best_params_)
print("\n") 
'''
FINALMENTE SE REALIZA TODO EL PROCESO DE EVALUACIÓN DEL CLASIFICADOR APLICADO 
QUE EN ESTE CASO, ES MÁQUINAS DE SOPORTE VECTORIAL.
IMPLEMENTANDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
'''
y_test_predicted = clf.predict(X_test)
y_test_scores = clf.decision_function(X_test)
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
print("\n") 

# IMPRESIÓN DE UN DATAFRAME CON LOS RESULTADOS.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")

Training time: 3.962164878845215s
Mejores Hiperparámetros: {'C': 10, 'decision_function_shape': 'ovo', 'kernel': 'rbf'}


Resultado mediante MCC: 0.9341319711428863
Resultado mediante Accuracy 0.9709302325581395


Reporte de resultados en cada clase:



,0,1,2,macro avg,weighted avg
precision,0.983740,0.882353,0.96875,0.944948,0.971520
recall,0.975806,0.937500,0.96875,0.960685,0.970930
f1-score,0.979757,0.909091,0.96875,0.952533,0.971136
support,124.000000,16.000000,32.00000,172.000000,172.000000


**KNN - K VECINOS MÁS CERCANOS**

In [ ]:
# SELECCIÓN DE HIPERPARÁMETROS DEL CLASIFICADOR. ESTOS SERÁN LOS QUE SE VAN A 
# ITERARSE.
estimator_KNN = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': (1,5,10,15,20,25,30),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')}

# AQUÍ SE PRE AJUSTA LA BÚSQUEDA POR REJILLA MEDIANTE VALIDACIÓN CRUZADA.
grid_search_KNN = GridSearchCV(estimator_KNN, param_grid=parameters_KNN, n_jobs = -1, cv = 5)
# SE ENTRENA EL MEJOR MODELO ARROJADO.
start = time.time()
grid_search_KNN.fit(X_train, Y_train)
stop = time.time()
print(f"Training time: {stop - start}s")

# IMPRESIÓN DE LOS RESULTADOS PARA CADA CLASE Y LAS CARACTERÍSTICAS DEL MODELO
# CON MEJORES HIPERPARÁMETROS.
print("Mejores Hiperparámetros:", grid_search_KNN.best_params_)
print("\n")
'''
FINALMENTE SE REALIZA TODO EL PROCESO DE EVALUACIÓN DEL CLASIFICADOR APLICADO 
QUE EN ESTE CASO, ES KNN O K VECINOS MÁS CERCANOS.
IMPLEMENTANDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
'''
y_test_predicted = grid_search_KNN.predict(X_test)
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
print("\n")

# IMPRESIÓN DE UN DATAFRAME CON LOS RESULTADOS.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")

Training time: 9.639192342758179s
Mejores Hiperparámetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}


Resultado mediante MCC: 0.8921036166748493
Resultado mediante Accuracy 0.9534883720930233


Reporte de resultados en cada clase:



,0,1,2,macro avg,weighted avg
precision,0.960630,0.8125,1.00000,0.924377,0.954175
recall,0.983871,0.8125,0.90625,0.900874,0.953488
f1-score,0.972112,0.8125,0.95082,0.911810,0.953303
support,124.000000,16.0000,32.00000,172.000000,172.000000


**NAIVE BAYES GAUSIANO**

In [ ]:
# AQUÍ SE IMPORTA LA RESPECTIVA LIBRERÍA DE NAIVE BAYES GAUSSIANO.
from sklearn.naive_bayes import GaussianNB
# SE GUARDA EL PARÁMETRO DEL CLASIFICADOR EN UNA VARIABLE.
clf_NB = GaussianNB()
# SE ENTRENA EL CLASIFICADOR.
start = time.time()
clf_NB.fit(X_train, Y_train)
stop = time.time()
print(f"Training time: {stop - start}s")
# SE IMPRIMEN LOS RESULTADOS DE RENDIMIENTO DEL CLASIFICADOR.
'''
AHORA SE HACE TODO EL PROCESO DE EVALUACIÓN DEL CLASIFICADOR APLICADO 
QUE EN ESTE CASO, ES NAIVE BAYES.
IMPLEMENTANDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
'''
y_test_predicted = clf_NB.predict(X_test)
print("\n")
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
print("\n")

# IMPRESIÓN DE UN DATAFRAME CON LOS RESULTADOS.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")


Training time: 0.006891489028930664s


Resultado mediante MCC: 0.8710461132776235
Resultado mediante Accuracy 0.9418604651162791


Reporte de resultados en cada clase:



,0,1,2,macro avg,weighted avg
precision,0.975207,0.684211,0.96875,0.876056,0.946936
recall,0.951613,0.812500,0.96875,0.910954,0.941860
f1-score,0.963265,0.742857,0.96875,0.891624,0.943783
support,124.000000,16.000000,32.00000,172.000000,172.000000


**REDES NEURONALES**

In [ ]:
# SE GUARDA EL CLASIFICADOR EN UNA VARIABLE DENOMINADA COMO MLP.
mlp = MLPClassifier(max_iter = 270, random_state=0)

# GUARDAMOS LOS HIPERPARÁMETROS A ITERAR.
espacio_parametros = {
    'hidden_layer_sizes': [(10,20), (20,30), (10,15,20), (5,10,15,20)],
    'activation': ['relu','sigmoid', 'logistic'],
    'solver': ['adam','sgd','lbfgs'],
    'alpha': [0.0001, 0.05, 0.01],
    'learning_rate': ['constant','adaptive'],
}

# SE REALIZA LA BUSQUEDA POR REJILLA MEDIANTE LA VALIDACIÓN CRUZADA.
mlp_clf = GridSearchCV(mlp, espacio_parametros, n_jobs=-1, cv=5)
# SE ENTRENA EL MODELO CON MEJOR RESULTADO.
start = time.time()
mlp_clf.fit(X_train, Y_train)
stop = time.time()
print(f"Training time: {stop - start}s")

Training time: 246.2027108669281s


In [ ]:
# IMPRESIÓN DE LOS RESULTADOS PARA CADA CLASE Y LAS CARACTERÍSTICAS DEL MODELO
# CON MEJORES HIPERPARÁMETROS.
print("Mejores Hiperparámetros:", mlp_clf.best_params_)
print("\n")
'''
FINALMENTE SE REALIZA TODO EL PROCESO DE EVALUACIÓN DEL CLASIFICADOR APLICADO 
QUE EN ESTE CASO, MULTI LAYER PERCEPTRON O RED NEURONAL MULTI CAPA (PERCEPTRÓN).
IMPLEMENTANDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
'''
y_test_predicted = mlp_clf.predict(X_test)
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
print("\n")

# IMPRESIÓN DE UN DATAFRAME CON LOS RESULTADOS.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")

Mejores Hiperparámetros: {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (20, 30), 'learning_rate': 'constant', 'solver': 'lbfgs'}


Resultado mediante MCC: 0.9352847301199171
Resultado mediante Accuracy 0.9709302325581395


Reporte de resultados en cada clase:



,0,1,2,macro avg,weighted avg
precision,0.991803,0.789474,1.000000,0.927092,0.974507
recall,0.975806,0.937500,0.968750,0.960685,0.970930
f1-score,0.983740,0.857143,0.984127,0.941670,0.972035
support,124.000000,16.000000,32.000000,172.000000,172.000000
